# TiDB Vector SDK V2

A powerful Python SDK for vector storage and retrieval operations with TiDB.

- 🔄 Automatic embedding generation
- 🔍 Vector similarity search
- 🎯 Advanced filtering capabilities
- 📦 Bulk operations support

## Installation

In [ ]:
%pip install autoflow-ai==0.0.1.dev10
%pip install dotenv ipywidgets pymysql sqlmodel

#### Configure environment variable

Go [tidbcloud.com](http://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster

Configuration can be provided through environment variables, or using `.env`:

In [ ]:
# Create .env file, then edit your .env, for example:
# $ cat .env
# DATABASE_URL='mysql+pymysql://root@localhost:4000/test'
# OPENAI_API_KEY='your_openai_api_key'
%cp .env.example .env

## Quickstart

### Connect to TiDB

In [3]:
import os
from autoflow.storage.tidb import TiDBClient

# Format: mysql+pymysql://<username>:<password>@<host>:4000/<database>
db = TiDBClient.connect(os.getenv("DATABASE_URL"))

### Create table

In [4]:
from typing import Optional, Any
from autoflow.storage.tidb.base import TiDBModel
from sqlmodel import Field
from autoflow.llms.embeddings import EmbeddingFunction

# Define your embedding model.
text_embed = EmbeddingFunction("openai/text-embedding-3-small")

class Chunk(TiDBModel, table=True):
    __tablename__ = "chunks"
    id: int = Field(primary_key=True)
    text: str = Field()
    text_vec: Optional[Any] = text_embed.VectorField(source_field="text")
    user_id: int = Field()

table = db.create_table(schema=Chunk)

### Insert Data

In [5]:
table.insert(Chunk(id=1, text="The quick brown fox jumps over the lazy dog", user_id=1),)
table.bulk_insert([
    Chunk(id=2, text="A quick brown dog runs in the park", user_id=2),
    Chunk(id=3, text="The lazy fox sleeps under the tree", user_id=2),
    Chunk(id=4, text="A dog and a fox play in the park", user_id=3)
])

[Chunk(user_id=2, id=2, text='A quick brown dog runs in the park', text_vec=array([-0.0412815 , -0.00934362,  0.01239674, ..., -0.00587278,
        -0.00735941,  0.01383422], dtype=float32)),
 Chunk(user_id=2, id=3, text='The lazy fox sleeps under the tree', text_vec=array([-0.01610469, -0.00269681, -0.01787939, ..., -0.00041015,
         0.01320426,  0.02987844], dtype=float32)),
 Chunk(user_id=3, id=4, text='A dog and a fox play in the park', text_vec=array([-2.7123539e-02, -4.4581316e-02, -3.8457386e-02, ...,
        -1.1360981e-03,  9.5597192e-05,  3.4092940e-02], dtype=float32))]

### Vector Search

In [8]:
chunks = table.search("A quick fox in the park").limit(3).to_pydantic()
[
    (c.id,  c.text, c.score)
    for c in chunks
]

[(4, 'A dog and a fox play in the park', 0.7308190419242949),
 (2, 'A quick brown dog runs in the park', 0.665493189763966),
 (1, 'The quick brown fox jumps over the lazy dog', 0.6157064668170177)]

### Advanced Filtering

TiDB Client supports various filter operators for flexible querying:

| Operator | Description | Example |
|----------|-------------|---------|
| `$eq` | Equal to | `{"field": {"$eq": "hello"}}` |
| `$gt` | Greater than | `{"field": {"$gt": 1}}` |
| `$gte` | Greater than or equal | `{"field": {"$gte": 1}}` |
| `$lt` | Less than | `{"field": {"$lt": 1}}` |
| `$lte` | Less than or equal | `{"field": {"$lte": 1}}` |
| `$in` | In array | `{"field": {"$in": [1, 2, 3]}}` |
| `$nin` | Not in array | `{"field": {"$nin": [1, 2, 3]}}` |
| `$and` | Logical AND | `{"$and": [{"field1": 1}, {"field2": 2}]}` |
| `$or` | Logical OR | `{"$or": [{"field1": 1}, {"field2": 2}]}` |


In [11]:
chunks = table.query({"user_id": 1})
[
    (c.id,  c.text, c.user_id)
    for c in chunks
]

[(1, 'The quick brown fox jumps over the lazy dog', 1)]

### Truncate table

In [12]:
table.truncate()